# Visualizing SGD, Momentum, Adagrad and RMSProp

# 可视化随机梯度下降，动量机制，适配学习率和RMSProp

By LongGang Pang@UC Berkeley

使用 Jupyter Notebook 里的 ipywidgets 插件，演示者可以非常方便的生成交互动画。

使用ipywidgets中的interact功能，我制作了这个随机梯度下降各种算法的动态演示动画。
用户可通过手动调节学习率，以及各种梯度下降独有的参数，观察这些算法在不同参数配置下的表现。

原静态示例来源于 Li Mu 以及 Alex 的 UC Berkeley 2019春季[深度学习课程课件](https://github.com/d2l-ai/berkeley-stat-157/blob/master/slides/5_2/momentum.ipynb)。特此感谢。

运行方法：
1. 在 Readme 里点击 Binder 按钮在线打开此 Notebook （不需本地安装）。或本地打开（需要安装Python环境及Jupyter Notebook）。
2. 点击代码块，按 Shift+Enter 即可运行相应代码。

In [14]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker, cm
import seaborn as sns
from ipywidgets import *
import math

sns.set_context('paper', font_scale=2)
sns.set_style('ticks')

下面这个Cell定义了4个函数，

1. f_2d(x1, x2) 定义了一个简单的 2 维函数，$f(x_1, x_2) = 0.1 x_1^2 + 2x_2^2$. 我们的目标是使用这个函数测试各种梯度下降算法，寻找极小值点。
2. f_grad(x1, x2) 计算 f_2d() 沿两个方向的梯度
3. train_2d() 函数使用给用的梯度下降算法，迭代50步，返回更新历史
4. plot_2d() 函数画出 f_2d 函数的等高图以及50步更新轨迹

In [15]:
def f_2d(x1, x2):
    '''original function to minimize'''
    return 0.1 * x1 ** 2 + 2 * x2 ** 2

def f_grad(x1, x2):
    '''the gradient dfdx1 and dfdx2'''
    dfdx1 = 0.2 * x1
    dfdx2 = 4 * x2
    return dfdx1, dfdx2

def train_2d(trainer, lr):
    """Train a 2d object function with a customized trainer"""
    x1, x2 = -5, -2
    s_x1, s_x2 = 0, 0
    res = [(x1, x2)]
    for i in range(50):
        x1, x2, s_x1, s_x2, lr = trainer(x1, x2, s_x1, s_x2, lr)
        res.append((x1, x2))
    return res

def plot_2d(res, figsize=(10, 6), title=None):
    x1_, x2_ = zip(*res)
    fig = plt.figure(figsize=figsize)
    plt.plot([0], [0], 'r*', ms=15)
    plt.text(0.0, 0.25, 'minimum', color='w')
    plt.plot(x1_[0], x2_[0], 'ro', ms=10)
    plt.text(x1_[0]+0.1, x2_[0]+0.2, 'start', color='w')
    plt.plot(x1_, x2_, '-o', color='#ff7f0e')
    
    plt.plot(x1_[-1], x2_[-1], 'wo')
    plt.text(x1_[-1], x2_[-1]-0.25, 'end', color='w')
    
    x1 = np.linspace(-5.5, 3, 50)
    x2 = np.linspace(min(-3.0, min(x2_) - 1), max(3.0, max(x2_) + 1), 100)
    x1, x2 = np.meshgrid(x1, x2)
    plt.contourf(x1, x2, f_2d(x1, x2), cmap=cm.gnuplot)
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title(title)
    plt.show()

## 机器学习原理

机器学习是一种新的编程模式。这种模式先假设任何输入 x 与输出 y 之间存在连续几何变换，
其函数形式可用 $y= f(x, \theta)$ 表示。其中 $\theta$ 是函数参数，初始化为随机数。
注意，初始化过程并不任意，这里有一篇很好的文章讲参数初始化 
[deeplearning.ai](http://www.deeplearning.ai/ai-notes/initialization/?utm_source=email&utm_medium=newsletter&utm_campaign=BlogAINotesInTextMay082019)。

机器学习通过逐步微调函数参数 $\theta$的方式，试图最小化模型预言 $\hat{y}$ 与真实标签 $y$的差别。
此差别最简单的数学表达式是 MAE 或者 MSE，即
$$L(\theta) = |\hat{y} - y|$$
或
$$L(\theta) = |\hat{y} - y|^2$$

为了最小化误差 $L(\theta)$, 我们只需要计算误差对参数 $\theta$ 的梯度，通过链式规则，
将误差反向传递到 $\theta$, 并使用梯度下降，更新$\theta$ 即可。

如果增加 $\theta$, 损失函数 $L(\theta)$ 增大，那么损失函数对参数$\theta$的梯度为正数, 即 $\nabla_{\theta} L(\theta) > 0$,
只要将 $\theta$ 减去一个正数 $\eta \cdot \nabla_{\theta} L(\theta)$ 即可使损失函数减小。
 其中 $\eta$ 是学习率 (Learning Rate, 简写为 lr)，为一个很小的正数。
 
如果增加 $\theta$, 损失函数 $L(\theta)$ 变小，那么 $\nabla_{\theta} L(\theta) < 0$,
将 $\theta$ 减去一个负数 $\eta \cdot \nabla_{\theta} L$ 同样可使损失函数减小。
你发现无论梯度是正是负，只要将参数减去学习率乘以梯度，总可以将损失降低。

这就是梯度下降的原理！

接下来会以交互式可视化的方式，展示几种最常用的梯度下降算法。

## 原始的梯度下降算法



$$\theta = \theta - \eta \cdot \nabla_{\theta} L(\theta)$$

In [16]:
def sgd(x1, x2, s1, s2, lr):
    dfdx1, dfdx2 = f_grad(x1, x2)
    return (x1 - lr * dfdx1, x2 - lr * dfdx2, 0, 0, lr)

@interact(lr=(0, 1, 0.001))
def visualize_gradient_descent(lr=0.05):
    res = train_2d(sgd, lr)
    plot_2d(res, title='SGD')


## 带动量的梯度下降算法


$$v_t = \gamma v_{t-1} + \eta \cdot \nabla_{\theta} L(\theta)$$
$$\theta = \theta - v_t$$

In [17]:
@interact(lr=(0, 0.99, 0.001), gamma=(0, 0.99, 0.001),
         continuous_update=False)
def visualize_sgd_momentum(lr=0.1, gamma=0.1):
    '''lr: learning rate
    gamma: parameter for momentum sgd'''
    
    def momentum(x1, x2, v1, v2, lr):
        dfdx1, dfdx2 = f_grad(x1, x2)
        v1 = gamma * v1 + lr * dfdx1
        v2 = gamma * v2 + lr * dfdx2
        x1 = x1 - v1
        x2 = x2 - v2
        return (x1, x2, v1, v2, lr)
    
    res = train_2d(momentum, lr)
    plot_2d(res, title='momentum')

## 带惯性的梯度下降算法

这是我自己测试的加入惯性的随机梯度下降算法。

$$v_t = \gamma v_{t-1} + (1 - \gamma) \cdot \nabla_{\theta} L(\theta)$$
$$\theta = \theta - \eta v_t$$

In [18]:
@interact(lr=(0, 0.99, 0.01), gamma=(0, 0.99, 0.01),
          continuous_update=False)
def visualize_sgd_inertia(lr=0.1, gamma=0.1):
    '''lr: learning rate
    gamma: parameter for inertia sgd'''
    
    def inertia(x1, x2, v1, v2, lr):
        dfdx1, dfdx2 = f_grad(x1, x2)
        v1 = gamma * v1 + (1 - gamma) * dfdx1
        v2 = gamma * v2 + (1 - gamma) * dfdx2
        x1 = x1 - lr * v1
        x2 = x2 - lr * v2
        return (x1, x2, v1, v2, lr)
    
    res = train_2d(inertia, lr)
    plot_2d(res, title='inertia')

## Adagrad 自适应调整的梯度下降

$$ g_t =  \nabla_{\theta} L(\theta)$$

$$ G = \sum_{t} g_t^2$$

$$\theta = \theta - \frac{\eta}{\sqrt{G + \epsilon}} \cdot g_t$$

In [20]:
@interact(lr=(0, 4, 0.01),
          continuous_update=False)
def visualize_adagrad(lr=0.1):
    '''lr: learning rate'''
    def adagrad_2d(x1, x2, s1, s2, lr):
        g1, g2 = f_grad(x1, x2)
        eps = 1e-6
        s1 += g1 ** 2
        s2 += g2 ** 2
        x1 -= lr / math.sqrt(s1 + eps) * g1
        x2 -= lr / math.sqrt(s2 + eps) * g2
        return x1, x2, s1, s2, lr
    
    res = train_2d(adagrad_2d, lr)
    plot_2d(res, title='adagrad')

## RMSProp 

$$ g = \nabla_{\theta} L(\theta) $$

$$ E\left[g^2\right] = \gamma E\left[g^2\right] + (1-\gamma) g^2 $$

$$\theta = \theta - \frac{\eta}{\sqrt{E\left[g^2\right] + \epsilon}} \cdot g$$

In [9]:
@interact(lr=(0, 1, 0.001), 
          gamma=(0, 0.99, 0.001),
          continuous_update=False)
def visualize_rmsprop(lr=0.1, gamma=0.9):
    '''lr: learning rate, 
       gamma: momentum'''  
    def rmsprop_2d(x1, x2, s1, s2, lr):
        eps = 1e-6
        g1, g2 = f_grad(x1, x2)
        s1 = gamma * s1 + (1 - gamma) * g1 ** 2
        s2 = gamma * s2 + (1 - gamma) * g2 ** 2
        x1 -= lr / math.sqrt(s1 + eps) * g1
        x2 -= lr / math.sqrt(s2 + eps) * g2
        return x1, x2, s1, s2, lr

    res = train_2d(rmsprop_2d, lr)
    plot_2d(res, title='rmsprop')

## Adam

$$ g = \nabla_{\theta} L(\theta) $$

$$ m = \beta_1 m + (1 - \beta_1) g $$

$$ n = \beta_2 n + (1 - \beta_2) g^2 $$

$$ \hat{m} = \frac{m}{(1 - \beta_1^t)} $$

$$ \hat{n} = \frac{n}{(1 - \beta_2^t)} $$

$$\theta = \theta - \frac{\eta}{\sqrt{\hat{n}} + \epsilon} \hat{m}$$

In [21]:
@interact(lr=(0, 1, 0.001), 
          beta1=(0, 0.999, 0.001),
          beta2=(0, 0.999, 0.001),
          continuous_update=False)
def visualize_adam(lr=0.1, beta1=0.9, beta2=0.999):
    '''lr: learning rate
    beta1: parameter for E(g)
    beta2: parameter for E(g^2)
    '''  
    def Deltax(m, n, g, t):
        eps = 1.0E-6
        m = beta1 * m + (1 - beta1) * g
        n = beta2 * n + (1 - beta2) * g*g
        m_hat = m / (1 - beta1**t)
        n_hat = n / (1 - beta2**t)
        dx = lr * m_hat / (math.sqrt(n_hat) + eps)
        return m, n, dx
        
    def adam_2d(x1, x2, m1, n1, m2, n2, lr, t):
        '''m1, m2: E(g1), E(g2)
           n1, n2: E(g1^2), E(g2^2) where E() is expectation
           lr: learning rate
           t: time step'''
        eps = 1e-6
        g1, g2 = f_grad(x1, x2)
        m1, n1, dx1 = Deltax(m1, n1, g1, t)
        m2, n2, dx2 = Deltax(m2, n2, g2, t)       
        x1 -= dx1
        x2 -= dx2
        return x1, x2, m1, n1, m2, n2, lr
    
    def train_adam(trainer, lr):
        """Train a 2d object function with a customized trainer"""
        x1, x2 = -5, -2
        m1, n1, m2, n2 = 0, 0, 0, 0
        res = [(x1, x2)]
        for i in range(30):
            x1, x2, m1, n1, m2, n2, lr = trainer(x1, x2, m1, n1, m2, n2, lr, i+1)
            res.append((x1, x2))
        return res
    
    res = train_adam(adam_2d, lr)
    plot_2d(res, title='adam')